In [9]:
!pip install pdfminer       
!pip install pdfminer.six  

^C
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140104 sha256=e8c735e8ce53f87dd235230fbd7590bbe6168732a083396bec652208ffcf169c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\1c\28\7d\f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer


In [16]:
!pip install packaging==20.0

  Attempting uninstall: packaging
    Found existing installation: packaging 18.0
    Uninstalling packaging-18.0:
      Successfully uninstalled packaging-18.0


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pyresparser 1.0.6 requires jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
en-core-web-sm 2.3.1 requires spacy<2.4.0,>=2.3.0, but you'll have spacy 3.2.1 which is incompatible.
en-core-web-lg 2.3.1 requires spacy<2.4.0,>=2.3.0, but you'll have spacy 3.2.1 which is incompatible.


In [10]:
!pip install doc2text

In [12]:
import io

In [17]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

# calling above function and extracting text
text = ''
for page in extract_text_from_pdf('D:/Daksh--Resume.pdf'):
    text += ' ' + page

In [15]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

AttributeError: 'Version' object has no attribute 'major'

In [58]:
extract_name(text)

'dakshpaleria |'

In [59]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [60]:
extract_mobile_number(text)

'7016397736'

In [61]:
def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [62]:
extract_email(text)

'dpaleria@gmail.com'

In [63]:
import pandas as pd
import spacy

# load pre-trained model
noun_chunks = []
nlp = spacy.load('en_core_web_sm')
for chunk in nlp(text).noun_chunks:
    noun_chunks.append(chunk.text)

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("D:/ResumeParser-master/resume_parser/resume_parser/skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in noun_chunks:
        token = token.lower().strip()
        if token in skills:
            skillset.append(token)
    if len(skillset)>20:
        skillset= skillset[0:30]
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

In [64]:
extract_skills(text)

['Javascript',
 'Ios',
 'Java',
 'Design',
 'Database',
 'Sql',
 'Php',
 'Tensorflow',
 'Flask',
 'Circuits',
 'Editing',
 'Python',
 'Android',
 'R',
 'Html',
 'Engineering',
 'Css',
 'Mobile',
 'Coding',
 'Github',
 'Visual',
 'Technical',
 'Docker',
 'System']

In [65]:
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

In [66]:
extract_education(text)

['BE']

In [67]:
def resume_sum(text):
    print("NAME:",extract_name(text))
    print("PHONE No.:",extract_mobile_number(text))
    print("EMAIL:",extract_email(text))
    print("SKILLS:",extract_skills(text))
    print("EDUCATION:",extract_education(text))

In [68]:
resume_sum(text)

NAME: dakshpaleria |
PHONE No.: 7016397736
EMAIL: dpaleria@gmail.com
SKILLS: ['Javascript', 'Ios', 'Java', 'Design', 'Database', 'Sql', 'Php', 'Tensorflow', 'Flask', 'Circuits', 'Editing', 'Python', 'Android', 'R', 'Html', 'Engineering', 'Css', 'Mobile', 'Coding', 'Github', 'Visual', 'Technical', 'Docker', 'System']
EDUCATION: ['BE']
COLLEGE: Saurashtra University


In [74]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [75]:
text1= NER(text)

In [2]:
def print_repo(repo):
    # repository full name
    print("Full name:", repo.full_name)
    # repository description
    print("Description:", repo.description)
    print("Date of last push:", repo.pushed_at)
    # programming language
    print("Language:", repo.language)
    print("Number of stars:", repo.stargazers_count)
    print("-"*50)
    # repository content (files & directories)
    try:
        # repo license
        print("License:", base64.b64decode(repo.get_license().content.encode()).decode())
    except:
        pass

In [3]:
import base64
from github import Github
from pprint import pprint

# Github username
username = "wroetoshaw"
# pygithub object
g = Github()
# get that user by username
user = g.get_user(username)
for repo in user.get_repos():
    print_repo(repo)
    print("="*100)

Full name: wroetoshaw/Character-Controller
Description: Using three js to controller-3d model with 3rd person camera
Date of last push: 2021-11-17 16:17:28
Language: JavaScript
Number of stars: 0
--------------------------------------------------
Full name: wroetoshaw/DarkMedia
Description: Dark Media is a multimedia talent management company and creative outpost for the largest creators on the internet.
Date of last push: 2021-11-05 09:16:22
Language: HTML
Number of stars: 1
--------------------------------------------------
Full name: wroetoshaw/Fancy-Scrolling-website-mp4
Description: Fancy scrolling website using mp4 file with trigger points and end points
Date of last push: 2021-11-05 08:41:24
Language: HTML
Number of stars: 2
--------------------------------------------------
Full name: wroetoshaw/Fancy-scrolling-Zflip
Description: Recreating fancy scrolling(using 200 img with 20 fps) 
Date of last push: 2021-11-04 09:50:25
Language: JavaScript
Number of stars: 1
----------------

In [80]:
!pip install PyGithub

In [95]:
from scrape_linkedin import ProfileScraper

with ProfileScraper() as scraper:
    profile = scraper.scrape(user='austinoboyle')
print(profile.to_dict())

ModuleNotFoundError: No module named 'scrape_linkedin'

In [97]:
!pip install git+git://github.com/austinoboyle/scrape-linkedin-selenium.git

  Cloning git://github.com/austinoboyle/scrape-linkedin-selenium.git to c:\users\user\appdata\local\temp\pip-req-build-6c5n8ys6
  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Created wheel for scrape-linkedin: filename=scrape_linkedin-0.8.0-py2.py3-none-any.whl size=23875 sha256=1f150ae0a229ae67c9c91cad8f30b909e1e397f4af581dd17f853eed30b3d841
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-w_de2w2z\wheels\db\38\14\026a904c87e4e4606cb1111c95b5eb6c4e0aa922c2ac9e0d7c
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=89fa9a344a78de10937dacffbaab79efdbe4d1145f6d5d18779ef45a63a6b0ea
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built scrape-linkedin bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.11.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.7 which is incompatible.
